In [2]:
import os
import cv2
from pathlib import Path
import random
import torch
import glob
from shutil import copyfile

In [3]:
#Change disk directory
base_path = Path("G:/Dissertation/")
if(Path().cwd() != Path(r"G:\Dissertation")):
    os.chdir(base_path)

In [4]:
#Define data_paths
raw_data_path = Path("raw_data/Data examples/")
raw_visibility_path = raw_data_path / Path("1_Visibility/")
raw_quality_path = raw_data_path/ Path("2_Quality/")
dataset_path = Path('dataset/')

In [5]:
from data_preparation import split_data

data_dir = Path("small_split/")
split_size = [0.8,0.1,0.1]

#split_data(dataset_path, data_dir, split_size, num_img_class=1000)

In [6]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
from torchvision import transforms
#Create transform (in this case for the ResNet images are resized to 224x224 and transformed into Tensors)
data_transform = transforms.Compose([
  # Resize our images to 64x64
  transforms.Resize(size=(224, 224)),
  # Flip the images randomly on the horizontal (just to make it as independent from the position of the pipeline)
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.RandomVerticalFlip(p=0.5),
  # Turn the image into a torch.Tensor
  transforms.ToTensor() 
])

In [8]:
from dataloaders import create_dataloaders

#Paths criados pelo split_data
train_dir = data_dir / Path('train/')
validation_dir = data_dir / Path('validation/')
test_dir = data_dir / Path('test/')

BATCH_SIZE = 32 # in the ResNet is 128 but my GPU doesn't have enough memory for that

train_dataloader, validation_dataloader, test_dataloader, class_names = create_dataloaders(train_dir,test_dir,validation_dir,data_transform, 32)

In [9]:
# Get a single image batch
image_batch, label_batch = next(iter(train_dataloader))
image_batch.shape, label_batch.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [10]:
torch.cuda.empty_cache()

In [11]:
!nvidia-smi

Wed Feb 15 20:33:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.01       Driver Version: 516.01       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   70C    P2    31W /  N/A |     80MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
from engine import train
from ResNet50 import ResNet50
import torch.nn as nn

# Set number of epochs
NUM_EPOCHS = 5

model = ResNet50().to(device)

# Setup loss function and optimizer 
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=0.001)

# Start the timer
from timeit import default_timer as timer
start_time = timer() 

# Train model
model_results = train(model=model,
                    train_dataloader=train_dataloader,
                    validation_dataloader=validation_dataloader,
                    optimizer=optimizer,
                    loss_fn=loss_fn,
                    epochs=NUM_EPOCHS,
                    device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.7925 | train_acc: 0.6460 | test_loss: 1.1040 | test_acc: 0.5801
Epoch: 2 | train_loss: 0.5364 | train_acc: 0.7470 | test_loss: 0.8271 | test_acc: 0.5949
Epoch: 3 | train_loss: 0.5430 | train_acc: 0.7418 | test_loss: 1.9650 | test_acc: 0.5391
Epoch: 4 | train_loss: 0.4429 | train_acc: 0.7917 | test_loss: 6.0000 | test_acc: 0.3945
Epoch: 5 | train_loss: 0.4562 | train_acc: 0.7907 | test_loss: 0.3677 | test_acc: 0.8184
Total training time: 600.939 seconds
